In [1]:
import numpy as np
import pandas as pd
import NelsonSiegelFunctions as nsf
import matplotlib.pyplot as plt
from scipy.optimize import fmin, minimize
from scipy.stats import skew, kurtosis

In [2]:
def NelsonSiegelParams(x,y, l1 = 0):
    if l1 != 0:
        fp = lambda c, x: (c[0])+ (c[1]*((1- np.exp(-x/l1))/(x/l1)))+ (c[2]*((((1-np.exp(-x/l1))/(x/l1)))- (np.exp(-x/l1))))
        p0 = np.array([0.01,0.01,0.01])
    else:
        fp = lambda c, x: (c[0])+ (c[1]*((1- np.exp(-x/c[3]))/(x/c[3])))+ (c[2]*((((1-np.exp(-x/c[3]))/(x/c[3])))- (np.exp(-x/c[3]))))
        p0 = np.array([0.01,0.01,0.01,1.00])  
    # error function to minimize
    e = lambda p, x, y: ((fp(p,x)-y)**2).sum()
      
    return (minimize(e, p0, args=(x,y)).x)

def NelsonSiegelSvenssonParams(x,y, l1 = 0, l2 = 0):
    if l1 != 0 and l2 != 0:
        fp = lambda c, x: (c[0])+ (c[1]*((1- np.exp(-x/l1))/(x/l1)))+ (c[2]*((((1-np.exp(-x/l1))/(x/l1)))- (np.exp(-x/l1))))+ (c[3]*((((1-np.exp(-x/l2))/(x/l2)))- (np.exp(-x/l2))))
        p0 = np.array([0.01,0.01,0.01,0.01]) 
    else:
        fp = lambda c, x: (c[0])+ (c[1]*((1- np.exp(-x/c[4]))/(x/c[4])))+ (c[2]*((((1-np.exp(-x/c[4]))/(x/c[4])))- (np.exp(-x/c[4]))))+ (c[3]*((((1-np.exp(-x/c[5]))/(x/c[5])))- (np.exp(-x/c[5]))))
        p0 = np.array([0.01,0.01,0.01,0.01,1.00,1.00]) 
    # error function to minimize
    e = lambda p, x, y: ((fp(p,x)-y)**2).sum()   
    return (minimize(e, p0, args=(x,y)).x)

def getNSSParams(df, l1 = 0, l2 = 0):
    x = df.columns.values
    dic={}
    for index, row in df.iterrows():
        y = df.loc[index].values
        params = NelsonSiegelSvenssonParams(x,y,l1,l2)
        dic[index] = params
    return pd.DataFrame.from_dict(dic)

def getNSParams(df, l1 = 0, l2 = 0):
    x = df.columns.values
    dic={}
    for index, row in df.iterrows():
        y = df.loc[index].values
        params = NelsonSiegelSvenssonParams(x,y,l1,l2)
        dic[index] = params
    return pd.DataFrame.from_dict(dic)

def evalNelsonSiegelSvensson(t,params, l1=0, l2=0):
    c = params
    if l1!=0 and l2!=0:
        c[4]=l1
        c[5]=l2
    j = []
    for h in t:
        #print(len(j))
        j.append((c[0])+ (c[1]*((1- np.exp(-h/c[4]))/(h/c[4])))+ (c[2]*((((1-np.exp(-h/c[4]))/(h/c[4])))- (np.exp(-h/c[4]))))+ (c[3]*((((1-np.exp(-h/c[5]))/(h/c[5])))- (np.exp(-h/c[5])))))
    return j

def getPred(df,df_params, l1=0, l2=0):
    # for the future, pass in what function you want to use to evaluate error
    '''
    Inputs:
    df is the original dataframe containing the data
    df_params is the parameters containing the parameters used to predict
    
    Output:
    returns a dataframe of the predicted values formated like the original
    '''
    x = df.columns.values
    dic = {}
    for column in df_params:
        dic[column] = evalNelsonSiegelSvensson(x,df_params[column], l1, l2)
    predicted = pd.DataFrame.from_dict(dic).transpose()
    predicted.columns = x
    return predicted


In [3]:
df = pd.read_csv("2016 2018 BPAM Yield Matrix.csv", index_col = 0)
df.index = pd.to_datetime(df.index)
df.columns = pd.to_numeric(df.columns.values)
df= df[df.index > np.datetime64("2016-01-01")]
df.head()

,3,6,12,24,36,60,84,120,180,240,300,360
2016-01-04,2.475,2.509,2.587,2.913,3.280,3.532,4.222,4.262,4.754,4.834,4.997,5.198
2016-01-05,2.469,2.503,2.580,2.916,3.295,3.512,4.180,4.275,4.765,4.846,5.010,5.214
2016-01-06,2.482,2.516,2.594,2.945,3.340,3.577,4.192,4.299,4.753,4.797,4.914,5.061
2016-01-07,2.543,2.577,2.656,2.986,3.357,3.592,4.189,4.304,4.758,4.801,4.919,5.068
2016-01-08,2.601,2.636,2.715,2.998,3.314,3.572,4.185,4.298,4.770,4.832,4.973,5.149


In [4]:
l1=74.5
l2=52.78

params = getNSSParams(df, l1, l2)
daily_pred = getPred(df,params, 74, 52)
nsf.getPredMatError(df,daily_pred)

{3: 0.0307683697019203,
 6: 0.015903533481069525,
 12: 0.04040079157283009,
 24: 0.04672887341197643,
 36: 0.043316228618004715,
 60: 0.05658458464025586,
 84: 0.06285266893216031,
 120: 0.12228418134053623,
 180: 0.06026940907132226,
 240: 0.073633766636231,
 300: 0.03957806196816218,
 360: 0.03969084735616652}

In [ ]:
for i in range(0,3):
    plt.figure(figsize = (8,4))
    plt.scatter(params.columns,params.loc[i])
    